In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
import matplotlib.pyplot as plt

macro_cols = ["balance_trade", "balance_trade_growth", "eurrub", "average_provision_of_build_contract",
"micex_rgbi_tr", "micex_cbi_tr", "deposits_rate", "mortgage_value", "mortgage_rate",
"income_per_cap", "rent_price_4+room_bus", "museum_visitis_per_100_cap", "apartment_build"]

df_train = pd.read_csv("./train.csv", parse_dates=['timestamp'])
df_test = pd.read_csv("./test.csv", parse_dates=['timestamp'])
df_macro = pd.read_csv("./macro.csv", parse_dates=['timestamp'], usecols=['timestamp'] + macro_cols)

df_train.head()

/Users/balkhin/anaconda/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,...,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,price_doc
0,1,2011-08-20,43,27.0,4.0,NaN,NaN,NaN,NaN,NaN,...,9,4,0,13,22,1,0,52,4,5850000
1,2,2011-08-23,34,19.0,3.0,NaN,NaN,NaN,NaN,NaN,...,15,3,0,15,29,1,10,66,14,6000000
2,3,2011-08-27,43,29.0,2.0,NaN,NaN,NaN,NaN,NaN,...,10,3,0,11,27,0,4,67,10,5700000
3,4,2011-09-01,89,50.0,9.0,NaN,NaN,NaN,NaN,NaN,...,11,2,1,4,4,0,0,26,3,13100000
4,5,2011-09-05,77,77.0,4.0,NaN,NaN,NaN,NaN,NaN,...,319,108,17,135,236,2,91,195,14,16331452


In [2]:
#adding price per sq meter
df_train['price_sqm']=df_train['price_doc']/df_train['full_sq']

In [3]:
#Creating dummy vars for all districts
raioni = list(set(list(df_train['sub_area'])))
for x in raioni: df_train[x] = df_train['sub_area']==x    
for x in raioni: df_test[x] = df_test['sub_area']==x
    
df_test.drop(['sub_area'], axis=1, inplace=True)
df_train.drop(['sub_area'], axis=1, inplace=True)

In [4]:
#Creating dummy for elevator
df_train['elevator'] = df_train['max_floor'] > 5
df_test['elevator'] = df_test['max_floor'] > 5

In [5]:
#Creating dummy for last floor
df_train['last_floor'] = df_train['max_floor'] == df_train['floor']
df_test['last_floor'] = df_test['max_floor'] == df_test['floor']

In [6]:
# ylog will be log(1+y), as suggested by https://github.com/dmlc/xgboost/issues/446#issuecomment-135555130
ylog_train_all = np.log1p(df_train['price_doc'].values)
id_test = df_test['id']

df_train.drop(['id', 'price_doc'], axis=1, inplace=True)
df_test.drop(['id'], axis=1, inplace=True)

# Build df_all = (df_train+df_test).join(df_macro)
num_train = len(df_train)
df_all = pd.concat([df_train, df_test])
df_all = pd.merge_ordered(df_all, df_macro, on='timestamp', how='left')
print(df_all.shape)

# Add month-year - не понял, надо ли именно так делать
month_year = (df_all.timestamp.dt.month + df_all.timestamp.dt.year * 100)
month_year_cnt_map = month_year.value_counts().to_dict()
df_all['month_year_cnt'] = month_year.map(month_year_cnt_map)

# Add week-year count - не понял, надо ли именно так делать
week_year = (df_all.timestamp.dt.weekofyear + df_all.timestamp.dt.year * 100)
week_year_cnt_map = week_year.value_counts().to_dict()
df_all['week_year_cnt'] = week_year.map(week_year_cnt_map)

# Add month and day-of-week
df_all['month'] = df_all.timestamp.dt.month
df_all['dow'] = df_all.timestamp.dt.dayofweek

# Other feature engineering - это фигня имхо, убрал
#df_all['rel_floor'] = df_all['floor'] / df_all['max_floor'].astype(float)
df_all['rel_kitch_sq'] = df_all['kitch_sq'] / df_all['full_sq'].astype(float)

# Remove timestamp column (may overfit the model in train)
df_all.drop(['timestamp'], axis=1, inplace=True)

(38133, 451)


In [7]:
# Deal with categorical values - районы выделил отдельно в даммиз, ибо важная переменная
df_numeric = df_all.select_dtypes(exclude=['object'])
df_obj = df_all.select_dtypes(include=['object']).copy()

for c in df_obj:
    df_obj[c] = pd.factorize(df_obj[c])[0]

df_values = pd.concat([df_numeric, df_obj], axis=1)

In [8]:
# Convert to numpy values
X_all = df_values.values
print(X_all.shape)

# Create a validation set, with last 20% of data
num_val = int(num_train * 0.2)

X_train_all = X_all[:num_train]
X_train = X_all[:num_train-num_val]
X_val = X_all[num_train-num_val:num_train]
ylog_train = ylog_train_all[:-num_val]
ylog_val = ylog_train_all[-num_val:]

X_test = X_all[num_train:]

df_columns = df_values.columns

print('X_train_all shape is', X_train_all.shape)
print('X_train shape is', X_train.shape)
print('y_train shape is', ylog_train.shape)
print('X_val shape is', X_val.shape)
print('y_val shape is', ylog_val.shape)
print('X_test shape is', X_test.shape)

(38133, 455)
X_train_all shape is (30471, 455)
X_train shape is (24377, 455)
y_train shape is (24377,)
X_val shape is (6094, 455)
y_val shape is (6094,)
X_test shape is (7662, 455)


In [9]:
dtrain_all = xgb.DMatrix(X_train_all, ylog_train_all, feature_names=df_columns)
dtrain = xgb.DMatrix(X_train, ylog_train, feature_names=df_columns)
dval = xgb.DMatrix(X_val, ylog_val, feature_names=df_columns)
dtest = xgb.DMatrix(X_test, feature_names=df_columns)

In [10]:
xgb_params = {
    'eta': 0.05,
    'max_depth': 5,
    'subsample': 1.0,
    'colsample_bytree': 0.7,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'silent': 1
}

# Uncomment to tune XGB `num_boost_rounds`
partial_model = xgb.train(xgb_params, dtrain, num_boost_round=1000, evals=[(dval, 'val')],
                       early_stopping_rounds=20, verbose_eval=20)

num_boost_round = partial_model.best_iteration

[0]	val-rmse:14.473
Will train until val-rmse hasn't improved in 20 rounds.
[20]	val-rmse:5.21437
[40]	val-rmse:1.8844
[60]	val-rmse:0.683054
[80]	val-rmse:0.254659
[100]	val-rmse:0.103693
[120]	val-rmse:0.054546
[140]	val-rmse:0.040402
[160]	val-rmse:0.035441
[180]	val-rmse:0.032756
[200]	val-rmse:0.031099
[220]	val-rmse:0.029832
[240]	val-rmse:0.028715
[260]	val-rmse:0.027818
[280]	val-rmse:0.027206
[300]	val-rmse:0.026591
[320]	val-rmse:0.02617
[340]	val-rmse:0.025758
[360]	val-rmse:0.025391
[380]	val-rmse:0.025083
[400]	val-rmse:0.024806
[420]	val-rmse:0.024527
[440]	val-rmse:0.024364
[460]	val-rmse:0.024192
[480]	val-rmse:0.024059
[500]	val-rmse:0.023887
[520]	val-rmse:0.023742
[540]	val-rmse:0.023623
[560]	val-rmse:0.023539
[580]	val-rmse:0.023454
[600]	val-rmse:0.023322
[620]	val-rmse:0.023249
[640]	val-rmse:0.02313
[660]	val-rmse:0.023037
[680]	val-rmse:0.022971
[700]	val-rmse:0.022904
[720]	val-rmse:0.022826
[740]	val-rmse:0.022751
[760]	val-rmse:0.022684
[780]	val-rmse:0.0226

In [14]:
fig, ax = plt.subplots(1, 1, figsize=(8, 16))
xgb.plot_importance(partial_model, height=0.5, ax=ax)
# max_num_features=50 выдает ошибку, убрал его, но так не рисует

In [15]:
num_boost_round = partial_model.best_iteration

In [16]:
model = xgb.train(dict(xgb_params, silent=0), dtrain_all, num_boost_round=num_boost_round)

In [17]:
fig, ax = plt.subplots(1, 1, figsize=(8, 16))
xgb.plot_importance(model, height=0.5, ax=ax)
# max_num_features=50 выдает ошибку, убрал его, но так не рисует

In [21]:
ylog_pred = model.predict(dtest)
y_pred = np.exp(ylog_pred) - 1

df_sub = pd.DataFrame({'id': id_test, 'price_doc': y_pred})

df_sub.to_csv('sub.csv', index=False)


In [22]:
ylog_hat= model.predict(dtrain_all)
y_hat = np.exp(ylog_hat) - 1

df_sub = pd.DataFrame({'price_doc': y_hat})

df_sub.to_csv('y_hat.csv', index=False)